In [1]:
import itertools
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
import plotly.express as px

from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# Let's define some constants

datasets = [
    "kuhar",
    "motionsense",
    "uci",
    "wisdm",
    "realworld"
]


labels_activity = {
    0: "sit",
    1: "stand",
    2: "walk",
    3: "stair up",
    4: "stair down",
    5: "run",
    6: "stair up and down",
}

colors_map = {
    "plotly-blue": px.colors.qualitative.Plotly[0],
    "plotly-green": px.colors.qualitative.Plotly[2],
    "plotly-purple": px.colors.qualitative.Plotly[3],
    "plotly-orange": px.colors.qualitative.Plotly[4],
    "plotly-cyan": px.colors.qualitative.Plotly[5],
    "plotly-pink": px.colors.qualitative.Plotly[6],
    "plotly-lightgreen": px.colors.qualitative.Plotly[7],
    "plotly-lightpink": px.colors.qualitative.Plotly[8],
    "plotly-yellow": px.colors.qualitative.Plotly[9],
    "plotly-red": px.colors.qualitative.Plotly[1],
}

colors = list(colors_map.values())
markers = ["circle", "square", "diamond", "x", "triangle-up", "triangle-down", "pentagon"]

In [3]:
# Let's define some filters related to each of the columns

# --- Related to domain ---
def filter_domain_fft_only(df):
    return df[df["transforms"] == "fft"]

def filter_domain_time_only(df):
    return df.loc[df["transforms"] == "time"]


# --- Related to features ---
def filter_features_all(df):
    return df[
        df["in_use_features"].str.contains("accel-x")
        & df["in_use_features"].str.contains("accel-y")
        & df["in_use_features"].str.contains("accel-z")
        & df["in_use_features"].str.contains("gyro-x")
        & df["in_use_features"].str.contains("gyro-y")
        & df["in_use_features"].str.contains("gyro-z")
    ]

def filter_features_accelerometer_only(df):
    return df[
        df["in_use_features"].str.contains("accel-x")
        & df["in_use_features"].str.contains("accel-y")
        & df["in_use_features"].str.contains("accel-z")
        & ~df["in_use_features"].str.contains("gyro-x")
        & ~df["in_use_features"].str.contains("gyro-y")
        & ~df["in_use_features"].str.contains("gyro-z")
    ]

def filter_features_gyroscope_only(df):
    return df[
        ~df["in_use_features"].str.contains("accel-x")
        & ~df["in_use_features"].str.contains("accel-y")
        & ~df["in_use_features"].str.contains("accel-z")
        & df["in_use_features"].str.contains("gyro-x")
        & df["in_use_features"].str.contains("gyro-y")
        & df["in_use_features"].str.contains("gyro-z")
    ]

# --- Related to how reducer is executed ---
def filter_reducer_over_all(df):
    return df[df["reduce_on"] == "all"]

def filter_reducer_over_sensor(df):
    return df[df["reduce_on"] == "sensor"]

def filter_reducer_over_axis(df):
    return df[df["reduce_on"] == "axis"]

# --- Related to the estimator ---
def filter_estimator_rf(df):
    return df[df["estimator"].str.lower().str.contains("randomforest")]

def filter_estimator_svm(df):
    return df[df["estimator"].str.lower().str.contains("svm")]

def filter_estimator_knn(df):
    return df[df["estimator"].str.lower().str.contains("knn")]

# --- Related to the scaler ---
def filter_scaler_none(df):
    return df[df["scaler"] == "no scaler"]

def filter_scaler_minmax(df):
    return df[df["scaler"].str.lower().str.contains("minmaxscaler")]

def filter_scaler_std(df):
   return df[df["scaler"].str.lower().str.contains("standardscaler")]

# --- Related to the view ---
def filter_view_standardized_intra(df):
    return df[df["view"] == "standartized_intra_balanced"]

def filter_view_standardized(df):
    return df[df["view"] == "standartized_balanced"]

def filter_view_raw(df):
    return df[df["view"] == "raw_balanced"]    

# --- Some other useful filters ---
def filter_only_reducer_dataset_equals_train_dataset(df):
    return df[df["reducer_datasets"] == df["train_datasets"]]

def filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df):
    return df[(df["reducer_datasets"] == df["train_datasets"]) | (df["reducer_datasets"] == "")]

def filter_only_reducer_dataset_diferent_from_train(df):
    return df[(df["reducer_datasets"] != df["train_datasets"]) & (df["reducer_datasets"] != "")]

def filter_same_train_test(df):
    return df[df["train_datasets"] == df["test_datasets"]]

def filter_different_train_test(df):
    return df[df["train_datasets"] != df["test_datasets"]]

In [4]:
# Preprocessing steps
def rename_datasets(
    df, columns: List[str] = ("reducer_datasets", "train_datasets", "test_datasets")
):
    def rename_row(row):
        for col in columns:
            names = set()
            for name in row[col].split(","):
                name = name.strip()
                names.add(name.split(".")[0])
            row[col] = ", ".join(sorted(names))
        return row

    df = df.apply(rename_row, axis=1)
    return df

def add_view_name(df, new_column_name: str = "view"):
    df[new_column_name] = df["config_id"].apply(lambda x: "_".join(x.split("_")[:-1]))
    return df

def match_configs(df, new_column_name: str = "config_group"):
    group_no = 0
    for k, subdf in df.groupby(
        [
            "in_use_features",
            "scale_on",
            "reduce_on",
            "transforms",
            "scaler",
            "reducer",
            "umap components",
            "reducer_datasets",
            "train_datasets",
            "test_datasets",
            "estimator",
        ]
    ):
        if len(subdf) == 2:
            df.loc[subdf.index, new_column_name] = group_no
            group_no += 1
    return df

In [5]:
results_file = Path("results.csv")
results = pd.read_csv(results_file).fillna("")
# Lets transform and add some useful information
results = rename_datasets(results)
results = add_view_name(results)
results = match_configs(results)
results

,experiment_name,run_name,config_id,reduce_size,train_size,test_size,in_use_features,scale_on,reduce_on,transforms,scaler,reducer,umap components,reducer_datasets,train_datasets,test_datasets,estimator,accuracy (mean),accuracy (std),f1-score macro (mean),f1-score macro (std),f1-score weighted (mean),f1-score weighted (std),view,config_group
0,reducer_comb_1,execution1,standartized_intra_balanced_08609,0,1495,120,"gyro-x, gyro-y, gyro-z",train,sensor,,,umap-7,7,,wisdm,wisdm,randomforest-100,0.670833,2.770128e-02,0.667083,3.039176e-02,0.667083,3.039176e-02,standartized_intra_balanced,NaN
1,reducer_comb_1,execution1,standartized_intra_balanced_08609,0,1495,120,"gyro-x, gyro-y, gyro-z",train,sensor,,,umap-7,7,,wisdm,wisdm,KNN-5,0.541667,1.110223e-16,0.540794,0.000000e+00,0.540794,0.000000e+00,standartized_intra_balanced,NaN
2,reducer_comb_1,execution1,standartized_intra_balanced_08609,0,1495,120,"gyro-x, gyro-y, gyro-z",train,sensor,,,umap-7,7,,wisdm,wisdm,SVM-rbf-C1.0,0.583333,1.110223e-16,0.521068,0.000000e+00,0.521068,0.000000e+00,standartized_intra_balanced,NaN
3,reducer_comb_1,execution1,standartized_intra_balanced_00845,0,1794,144,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",train,sensor,,MinMaxScaler,umap-2,2,,kuhar,kuhar,randomforest-100,0.800694,1.812221e-02,0.795044,1.921549e-02,0.795044,1.921549e-02,standartized_intra_balanced,NaN
4,reducer_comb_1,execution1,standartized_intra_balanced_00845,0,1794,144,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",train,sensor,,MinMaxScaler,umap-2,2,,kuhar,kuhar,KNN-5,0.493056,5.551115e-17,0.482479,0.000000e+00,0.482479,5.551115e-17,standartized_intra_balanced,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63172,reducer_comb_1,execution1,raw_balanced_02926,3978,3978,1062,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",train,all,FFT-centered,MinMaxScaler,umap-20,20,motionsense,motionsense,motionsense,KNN-5,0.883239,1.110223e-16,0.883064,1.110223e-16,0.883064,0.000000e+00,raw_balanced,3678.0
63173,reducer_comb_1,execution1,raw_balanced_02926,3978,3978,1062,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",train,all,FFT-centered,MinMaxScaler,umap-20,20,motionsense,motionsense,motionsense,SVM-rbf-C1.0,0.870998,1.110223e-16,0.871440,1.110223e-16,0.871440,1.110223e-16,raw_balanced,3679.0
63174,reducer_comb_1,execution1,standartized_intra_balanced_05531,0,1794,144,"accel-x, accel-y, accel-z",train,sensor,,StandardScaler,umap-5,5,,motionsense,motionsense,randomforest-100,0.798611,1.889093e-02,0.796915,1.948198e-02,0.796915,1.948198e-02,standartized_intra_balanced,NaN
63175,reducer_comb_1,execution1,standartized_intra_balanced_05531,0,1794,144,"accel-x, accel-y, accel-z",train,sensor,,StandardScaler,umap-5,5,,motionsense,motionsense,KNN-5,0.562500,0.000000e+00,0.546329,0.000000e+00,0.546329,0.000000e+00,standartized_intra_balanced,NaN


In [6]:
results = results[
    ~(results["reducer_datasets"] == "")
    | (results["umap components"] == 0)
]
list(results.columns)

['experiment_name',
 'run_name',
 'config_id',
 'reduce_size',
 'train_size',
 'test_size',
 'in_use_features',
 'scale_on',
 'reduce_on',
 'transforms',
 'scaler',
 'reducer',
 'umap components',
 'reducer_datasets',
 'train_datasets',
 'test_datasets',
 'estimator',
 'accuracy (mean)',
 'accuracy (std)',
 'f1-score macro (mean)',
 'f1-score macro (std)',
 'f1-score weighted (mean)',
 'f1-score weighted (std)',
 'view',
 'config_group']

In [7]:
# Lets filter out some reduntant lines
results = results[
    ~(results["reducer_datasets"] == "")
    | (results["umap components"] == 0)
]

# As we only use umap, we can drop the reducer column
results = results.drop(columns=["reducer"])

# As we only use TIME and FFT domains, lets rename the values "" of column transforms to "TIME"
# and "FFT" to "FFT"
results.loc[results["transforms"] == "", "transforms"] = "time" 
results.loc[results["transforms"] == "FFT-centered", "transforms"] = "fft"

# If no scaler is used, lets change the value from "" to "no scaler"
results.loc[results["scaler"] == "", "scaler"] = "no scaler"

# As scaler is only used for the train dataset, we can drop the column scale_on
results = results.drop(columns=["scale_on"])

# Lets drop some meta-columns that we will not use
results = results.drop(columns=["experiment_name", "run_name", "config_id", "reduce_size", "train_size", "test_size"])
list(results.columns)

['in_use_features',
 'reduce_on',
 'transforms',
 'scaler',
 'umap components',
 'reducer_datasets',
 'train_datasets',
 'test_datasets',
 'estimator',
 'accuracy (mean)',
 'accuracy (std)',
 'f1-score macro (mean)',
 'f1-score macro (std)',
 'f1-score weighted (mean)',
 'f1-score weighted (std)',
 'view',
 'config_group']

In [8]:
# Grouping by this columns should result in only one row per group
# It is a unique tuple that describes each experiment
unique_exp_columns = [
    "in_use_features",
    "reduce_on",
    "transforms",
    "scaler",
    "umap components",
    "reducer_datasets",
    "train_datasets",
    "test_datasets",
    "estimator",
    "view",
]

# Sanity check
results.groupby(unique_exp_columns).apply(lambda x: len(x) == 1).value_counts()

True    54267
dtype: int64

In [9]:
def chart_bar_with_error_bars(df, improvement_col: str, groupby: str = "train_datasets"):
    xs, ys, error_y_min, error_y_max = [], [], [], []
    for key, subdf in df.groupby(groupby):
        sorted_improvement = sorted(subdf[improvement_col].values)
        # print(sorted_improvement)
        xs.append(key)
        ys.append(sorted_improvement[1])
        error_y_min.append(abs(sorted_improvement[0]-sorted_improvement[1]))
        error_y_max.append(abs(sorted_improvement[-1]-sorted_improvement[1]))

    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=xs,
            y=ys,
            error_y=dict(
                type="data",
                symmetric=False,
                array=error_y_max,
                arrayminus=error_y_min
            )
        )
    )
    return fig

def chart_bar_with_side_by_side(df, improvement_col: str, groupby: str = "train_datasets", inner_group: str = "estimator"):
    fig = go.Figure()

    for i, (key, subdf) in enumerate(df.groupby(groupby)):
        xs, ys = [], []
        for inner_key, inner_subdf in subdf.groupby(inner_group):
            xs.append(str(inner_key))
            ys.append(inner_subdf[improvement_col].values[0])
        
        fig.add_trace(
            go.Bar(
                y=ys,
                x=xs,
                name=str(key),
                marker=dict(color=colors[i])
            )
        )
    return fig

def best_of_each_estimator(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["estimator"]]

    best_of_each_dataset = []
    for key, subdf in df.groupby(experiment_uniq):
        best = subdf.sort_values(by="accuracy (mean)", ascending=False, inplace=False).iloc[0]
        best_of_each_dataset.append(best)
    return pd.DataFrame(best_of_each_dataset)

def best_of_each_umap(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["umap components"]]

    best_of_each_dataset = []
    for key, subdf in df.groupby(experiment_uniq):
        best = subdf.sort_values(by="accuracy (mean)", ascending=False, inplace=False).iloc[0]
        best_of_each_dataset.append(best)
    return pd.DataFrame(best_of_each_dataset)

def best_of_each_estimator_and_umap(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["estimator", "umap components"]]

    best_of_each_dataset = []
    for key, subdf in df.groupby(experiment_uniq):
        best = subdf.sort_values(by="accuracy (mean)", ascending=False, inplace=False).iloc[0]
        best_of_each_dataset.append(best)
    return pd.DataFrame(best_of_each_dataset)

## Pergunta 1

### a) Qual o impacto no desempenho dos modelos de ML quando o dado de entrada é transformado com o UMAP?

Compararemos, para cada tupla <dataset DS; domíno D; modelo de ML M>, qual é a diferença (razão) de desempenho do modelo M na tarefa de HAR com o dado do dataset DS no domínio D com e sem redução de dimensionalidade.

In [85]:
def get_umap_improvement_over_no_umap(df, new_column_name: str = "umap improvement"):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["reducer_datasets", "umap components"]]

    zero_umap = {}
    for _, subdf in df[df["umap components"] == 0].iterrows():
        zero_umap[tuple(subdf[experiment_uniq].values.tolist())] = subdf["accuracy (mean)"]

    df[new_column_name] = df.apply(lambda x: x["accuracy (mean)"] / zero_umap.get(tuple(x[experiment_uniq].values.tolist()), np.nan), axis=1)
    return df

def filter_max_set_only(df, column: str = "umap improvement"):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["reducer_datasets", "umap components"]]
    df = df.dropna(subset=[column], inplace=False)
    df = df[df["umap components"] > 0]
    lines = [subdf[subdf[column] == subdf[column].max()].iloc[0] for _, subdf in df.groupby(experiment_uniq)]
    df = pd.DataFrame(lines)
    return df

df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
print(results.shape)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)
df = get_umap_improvement_over_no_umap(df)
df = filter_max_set_only(df)

fig = chart_bar_with_error_bars(df, improvement_col="umap improvement", groupby="train_datasets")
fig.add_hline(y=1.0, line_dash="dash", line_color="magenta")
fig.update_layout(
    title="UMAP Improvement for each dataset (best UMAP over no UMAP, per dataset).<br>Error bars comes from different estimators",
    xaxis_title="Dataset",
    yaxis_title="Accuracy Improvement over no UMAP",
    width=800,
    height=600
)
fig.show()

fig = chart_bar_with_side_by_side(df, improvement_col="umap improvement", groupby="estimator", inner_group="train_datasets")
fig.add_hline(y=1.0, line_dash="dash", line_color="red")
fig.update_layout(
    title="UMAP Improvement for each dataset (best UMAP over no UMAP, per dataset)",
    xaxis_title="Dataset",
    yaxis_title="Accuracy Improvement over no UMAP",
    legend_title="Estimator",
    width=800,
    height=600
)
fig.show()

(54267, 17)


### b) Qual é o impacto da dimensionalidade alvo do UMAP no desempenho dos modelos de ML?

b .1) UMAP vs No-UMAP

In [11]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)
df = get_umap_improvement_over_no_umap(df)
df = df[df["umap components"] > 0]

dset_markers = {
    name: markers[i]
    for i, name in enumerate(df["train_datasets"].unique())
}

estimator_colors = {
    name: colors[i]
    for i, name in enumerate(df["estimator"].unique())
}

fig = go.Figure()
for (estimator, dataset), subdf in df.groupby(["estimator", "train_datasets"]):
    subdf = subdf.sort_values(by="umap components")
    xs, ys = [], []
    for n_components in sorted(subdf["umap components"].unique()):
        xs.append(n_components)
        ys.append(subdf[subdf["umap components"] <= n_components]["umap improvement"].max())
    fig.add_trace(
        go.Scatter( 
            x=xs,
            y=ys,
            name=f"{estimator} {dataset}",
            mode="lines+markers",
            legendgroup=estimator,
            # legendgrouptitle="Estimator",
            marker=dict(
                color=estimator_colors[estimator],
                symbol=dset_markers[dataset],
                size=8,
            ),
        )
    )
    # subdf["best umap improvement so far"] = df["umap improvement"].expanding().max()

fig.update_layout(
    title="UMAP Improvement for each dataset (best UMAP over no UMAP, per dataset)",
    xaxis_title="Maximum UMAP Components",
    yaxis_title="Accuracy improvement over no UMAP",
    legend_title="Estimator",
    width=1200,
    height=600,
    xaxis=dict(tickmode="linear")
)
fig.show()

b.2) UMAP vs best UMAP

In [12]:
def get_umap_improvement_over_best_umap(df, new_column_name: str = "umap improvement"):
    df = df[df["umap components"] != 0]
    experiment_uniq = [c for c in unique_exp_columns if c not in ["reducer_datasets", "umap components"]]

    best_umap = {}
    for key, subdf in df.groupby(experiment_uniq):
        best_umap[key] = subdf["accuracy (mean)"].max()

    for key, subdf in df.groupby(experiment_uniq):
        df.loc[subdf.index, new_column_name] = subdf["accuracy (mean)"] / best_umap[key]

    return df

In [13]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)
df = get_umap_improvement_over_best_umap(df)

dset_markers = {
    name: markers[i]
    for i, name in enumerate(df["train_datasets"].unique())
}

estimator_colors = {
    name: colors[i]
    for i, name in enumerate(df["estimator"].unique())
}

fig = go.Figure()
for (estimator, dataset), subdf in df.groupby(["estimator", "train_datasets"]):
    subdf = subdf.sort_values(by="umap components")
    xs, ys = [], []
    for n_components in sorted(subdf["umap components"].unique()):
        xs.append(n_components)
        ys.append(subdf[subdf["umap components"] <= n_components]["umap improvement"].max())
    fig.add_trace(
        go.Scatter( 
            x=xs,
            y=ys,
            name=f"{estimator} {dataset}",
            mode="lines+markers",
            legendgroup=estimator,
            # legendgrouptitle="Estimator",
            marker=dict(
                color=estimator_colors[estimator],
                symbol=dset_markers[dataset],
                size=7,
            ),
        )
    )
    # subdf["best umap improvement so far"] = df["umap improvement"].expanding().max()

fig.update_layout(
    title="UMAP Improvement for each dataset (UMAP over best UMAP, per dataset)",
    xaxis_title="Maximum UMAP Components",
    yaxis_title="Accuracy improvement best UMAP",
    legend_title="Estimator",
    width=1200,
    height=600,
    xaxis=dict(tickmode="linear")
)
fig.show()

/tmp/ipykernel_205650/3552989575.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Pergunta 1.c

In [14]:
experiment_uniq = [c for c in unique_exp_columns if c not in ["train_datasets", "view"]]

df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)

fig = go.Figure()


for i, (dset_name, train_df) in enumerate(df.groupby(["train_datasets"])):
    xs, ys = [], []
    for j, (view_name, subdf) in enumerate(train_df.groupby("view")):
        umap_0_max = subdf[subdf["umap components"] == 0]["accuracy (mean)"].max()
        xs.append(f"{dset_name} {view_name.split('_')[0]} (no umap)")
        ys.append(umap_0_max)
    
    fig.add_trace(
        go.Bar(
            x=xs,
            y=ys,
            name=f"{dset_name} no umap",
            marker=dict(color=colors[0]),
            legendgroup="no umap",
        )
    )
    xs, ys = [], []
    for j, (view_name, subdf) in enumerate(train_df.groupby("view")):
        umap_best_max = subdf[subdf["umap components"] > 0]["accuracy (mean)"].max()
        xs.append(f"{dset_name} {view_name.split('_')[0]} (best umap)")
        ys.append(umap_best_max)
    
    fig.add_trace(
        go.Bar(
            x=xs,
            y=ys,
            name=f"{dset_name} best umap",
            marker=dict(color=colors[1]),
            legendgroup="best umap",
        )
    )

fig.update_layout(
    title="Impact of standardization process on UMAP",
    xaxis_title="Dataset view",
    yaxis_title="Accuracy of best UMAP and best classifier",
    legend_title="Estimator",
    width=1200,
    height=800,
    xaxis=dict(tickangle=90)
)

# fig.update_xaxes(tickangle=75)
fig.show()

/tmp/ipykernel_205650/862335259.py:13: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



## Pergunta 2

Como a escolha do dataset no treinamento do UMAP afeta os resultados da tarefa?

In [30]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
# df = pd.concat([filter_only_reducer_dataset_diferent_from_train(df), filter_only_reducer_dataset_equals_train_dataset(df)])
df = filter_only_reducer_dataset_diferent_from_train(df)
df = filter_same_train_test(df)
df = best_of_each_estimator(df)

marker_colors = list(itertools.product(markers, colors))

fig = go.Figure()

for i, n_components in enumerate(sorted(df["umap components"].unique())):
    df_comp = df[df["umap components"] == n_components]
    # df_comp = best_of_each_estimator(df_comp)
    df_comp.sort_values(by="accuracy (mean)", inplace=True)
    fig.add_trace(
        go.Scatter(
            x=list(range(len(df_comp))),
            y=df_comp["accuracy (mean)"],
            name=f"UMAP {n_components} components",
            mode="lines+markers",
            marker=dict(
                color=marker_colors[i][1],
                symbol=marker_colors[i][0],
                size=7,
            ),
        )
    )

fig.update_layout(
    title="FFT",
    xaxis_title="Number of experiments",
    yaxis_title="Accuracy (best estimator only)",
    legend_title="UMAP Components",
    width=1200,
    height=600,
    yaxis=dict(range=[0, 1]),
    # xaxis=dict(tickmode="linear")
)
fig.show()

/tmp/ipykernel_205650/3671092761.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
df = results.copy()
df = filter_domain_time_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
# df = pd.concat([filter_only_reducer_dataset_diferent_from_train(df), filter_only_reducer_dataset_equals_train_dataset(df)])
df = filter_only_reducer_dataset_diferent_from_train(df)
df = filter_same_train_test(df)
df = best_of_each_estimator(df)

marker_colors = list(itertools.product(markers, colors))

fig = go.Figure()

for i, n_components in enumerate(sorted(df["umap components"].unique())):
    df_comp = df[df["umap components"] == n_components]
    # df_comp = best_of_each_estimator(df_comp)
    df_comp.sort_values(by="accuracy (mean)", inplace=True)
    fig.add_trace(
        go.Scatter(
            x=list(range(len(df_comp))),
            y=df_comp["accuracy (mean)"],
            name=f"UMAP {n_components} components",
            mode="lines+markers",
            marker=dict(
                color=marker_colors[i][1],
                symbol=marker_colors[i][0],
                size=7,
            ),
        )
    )

fig.update_layout(
    title="Time",
    xaxis_title="Number of experiments",
    yaxis_title="Accuracy (best estimator only)",
    legend_title="UMAP Components",
    width=1200,
    height=600,
    yaxis=dict(range=[0, 1]),
    # xaxis=dict(tickmode="linear")
)
fig.show()

/tmp/ipykernel_205650/2338358819.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Pergunta 3

## Pergunta 4

## Pergunta 5

É mais vantajoso reduzir a dimensionalidade dos dados dos sensores de forma individual (aplicando o UMAP a um sensor, ou eixo, por vez) ou de forma agregada (como estamos fazendo)?

In [17]:
def get_reduce_on_improvement(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["reduce_on", "umap components"]]

    all_values = {}
    for _, subdf in df[df["reduce_on"] == "all"].iterrows():
        all_values[tuple(subdf[experiment_uniq].values.tolist())] = subdf["accuracy (mean)"]

    df["umap improvement"] = df.apply(lambda x: x["accuracy (mean)"] / all_values.get(tuple(x[experiment_uniq].values.tolist()), np.nan), axis=1)
    df = df.dropna(subset=["umap improvement"], inplace=False)
    df = df[df["reduce_on"] != "all"]
    lines = [subdf[subdf["umap improvement"] == subdf["umap improvement"].max()].iloc[0] for _, subdf in df.groupby(experiment_uniq)]
    df = pd.DataFrame(lines)
    return df

# results = add_umap_improvement(results)
#df = only_standardized_view(no_scaler(using_all_features(only_fft(only_reduce_on_all(only_reducer_equals_train(df))))))

In [18]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
# df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = best_of_each_umap(df)
df = filter_only_reducer_dataset_equals_train_dataset(df)
df = get_reduce_on_improvement(df)


fig = chart_bar_with_side_by_side(df, improvement_col="umap improvement", groupby="estimator", inner_group=["train_datasets", "reduce_on"])
fig.update_layout(
    title="Improvement using multiple UMAPs to train per sensor and per axis over using all features",
    xaxis_title="reduce_on / dataset",
    yaxis_title="Improvement over using all features",
    legend_title="Estimator",
    width=1200,
    height=600
)
fig.show()


In [62]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
# df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = best_of_each_estimator(df)
df = filter_only_reducer_dataset_equals_train_dataset(df)
# df = get_reduce_on_improvement(df)

names = []

fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
for i, (reduce_on, subdf) in enumerate(df.groupby("reduce_on")):
    for j, (dataset_name, subsubdf) in enumerate(subdf.groupby("train_datasets")):
        subsubdf = subsubdf.sort_values(by="umap components", inplace=False)
        fig.add_trace(
            go.Scatter(
                x=subsubdf["umap components"],
                y=subsubdf["accuracy (mean)"],
                name=dataset_name,
                mode="lines+markers",
                marker=dict(color=colors[j]),
            ),
            row=1, col=i+1
        )
    names.append(reduce_on)

fig["layout"]["xaxis1"].update(title=f"UMAP components ({names[0]})", tickmode="linear")
fig["layout"]["xaxis2"].update(title=f"UMAP components ({names[1]})", tickmode="linear")
fig["layout"]["xaxis3"].update(title=f"UMAP components ({names[2]})", tickmode="linear")
fig["layout"]["yaxis1"].update(title="Accuracy (mean)", range=[0.5, 1])
fig["layout"]["yaxis1"].update(range=[0.5, 1])
fig["layout"]["yaxis1"].update(range=[0.5, 1])
fig.update_layout(
    title="UMAP applied to train per sensor and per axis and using all features",
    width=1200,
    height=600,
    legend_title="Dataset",
    showlegend=False,
    xaxis=dict(tickmode="linear"),
    yaxis=dict(range=[0, 1])

)
    
fig.show()

## Pergunta 6

Há uma mudança significativa entre os diferentes classificadores?

In [20]:
# def get_estimator_improvement(df):
#     experiment_uniq = [c for c in unique_exp_columns if c not in ["estimator"]]
#     df = df[df["umap components"] > 0]

#     all_values = {}
#     for _, subdf in df[df["reduce_on"] == "all"].iterrows():
#         all_values[tuple(subdf[experiment_uniq].values.tolist())] = subdf["accuracy (mean)"]

#     df["umap improvement"] = df.apply(lambda x: x["accuracy (mean)"] / all_values.get(tuple(x[experiment_uniq].values.tolist()), np.nan), axis=1)
#     df = df.dropna(subset=["umap improvement"], inplace=False)
#     df = df[df["reduce_on"] != "all"]
#     lines = [subdf[subdf["umap improvement"] == subdf["umap improvement"].max()].iloc[0] for _, subdf in df.groupby(experiment_uniq)]
#     df = pd.DataFrame(lines)
#     return df

# # results = add_umap_improvement(results)
# #df = only_standardized_view(no_scaler(using_all_features(only_fft(only_reduce_on_all(only_reducer_equals_train(df))))))

## Pergunta 7

Melhor no domínio da frequência ou tempo (condicionada à abordagem - manifold learning com UMAP)? (ou wavelet?)

In [90]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)
df = get_umap_improvement_over_no_umap(df)
df = filter_max_set_only(df)

fig = chart_bar_with_side_by_side(df, improvement_col="accuracy (mean)", groupby="estimator", inner_group="train_datasets")
# fig.add_hline(y=1.0, line_dash="dash", line_color="magenta")
fig.update_layout(
    title="Accuracy using FFT",
    xaxis_title="Dataset",
    yaxis_title="Accuracy (mean)",
    legend_title="Estimator",
    width=800,
    height=600
)
fig.show()

In [91]:
df = results.copy()
df = filter_domain_time_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
df = filter_only_reducer_dataset_equals_train_dataset_or_no_reducer(df)
df = get_umap_improvement_over_no_umap(df)
df = filter_max_set_only(df)

fig = chart_bar_with_side_by_side(df, improvement_col="accuracy (mean)", groupby="estimator", inner_group="train_datasets")
# fig.add_hline(y=1.0, line_dash="dash", line_color="magenta")
fig.update_layout(
     title="Accuracy using Time",
    xaxis_title="Dataset",
    yaxis_title="Accuracy (mean)",
    legend_title="Estimator",
    width=800,
    height=600
)
fig.show()

## Pergunta 8

## Pergunta 9

In [78]:
def get_scaler_improvement(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["scaler"]]

    all_values = {}
    for _, subdf in df[df["scaler"] == "no scaler"].iterrows():
        all_values[tuple(subdf[experiment_uniq].values.tolist())] = subdf["accuracy (mean)"]

    df["umap improvement"] = df.apply(lambda x: x["accuracy (mean)"] / all_values.get(tuple(x[experiment_uniq].values.tolist()), np.nan), axis=1)
    df = df.dropna(subset=["umap improvement"], inplace=False)
    df = df[df["scaler"] != "no scaler"]
    lines = [subdf[subdf["umap improvement"] == subdf["umap improvement"].max()].iloc[0] for _, subdf in df.groupby(experiment_uniq)]
    df = pd.DataFrame(lines)
    return df

In [93]:
df = results.copy()
df = filter_domain_fft_only(df)
df = filter_features_all(df)
df = filter_reducer_over_all(df)
# df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
# df = best_of_each_umap(df)
df = filter_only_reducer_dataset_equals_train_dataset(df)
df = get_scaler_improvement(df)

fig = chart_bar_with_side_by_side(df, improvement_col="umap improvement", groupby="estimator", inner_group=["train_datasets", "scaler"])
fig.add_hline(y=1.0, line_dash="dash", line_color="red")
fig.update_layout(
    title="Improvement using different scalers",
    xaxis_title="dataset / scaler",
    yaxis_title="Improvement over using no scaler",
    legend_title="Estimator",
    width=1200,
    height=600
)
fig.show()


## Pergunta 10

In [86]:
def get_in_use_features_improvement(df):
    experiment_uniq = [c for c in unique_exp_columns if c not in ["in_use_features"]]

    all_values = {}
    for _, subdf in df[(df["in_use_features"].str.contains("accel-x")) & (df["in_use_features"].str.contains("gyro-x"))  ].iterrows():
        all_values[tuple(subdf[experiment_uniq].values.tolist())] = subdf["accuracy (mean)"]

    df["umap improvement"] = df.apply(lambda x: x["accuracy (mean)"] / all_values.get(tuple(x[experiment_uniq].values.tolist()), np.nan), axis=1)
    df = df.dropna(subset=["umap improvement"], inplace=False)
    df = df.loc[
        ~(df["in_use_features"].str.contains("accel-y")
        & df["in_use_features"].str.contains("accel-z")
        & df["in_use_features"].str.contains("gyro-x")
        & df["in_use_features"].str.contains("gyro-y")
        & df["in_use_features"].str.contains("gyro-z"))
    ]
    lines = [subdf[subdf["umap improvement"] == subdf["umap improvement"].max()].iloc[0] for _, subdf in df.groupby(experiment_uniq)]
    df = pd.DataFrame(lines)
    return df

In [94]:
df = results.copy()
df = filter_domain_fft_only(df)
# df = filter_features_all(df)
df = filter_reducer_over_all(df)
df = filter_scaler_none(df)
df = filter_view_standardized_intra(df)
# df = best_of_each_umap(df)
df = filter_only_reducer_dataset_equals_train_dataset(df)
df = get_in_use_features_improvement(df)

fig = chart_bar_with_side_by_side(df, improvement_col="umap improvement", groupby="estimator", inner_group=["train_datasets", "in_use_features"])
fig.add_hline(y=1.0, line_dash="dash", line_color="red")
fig.update_layout(
    title="Improvement using different features",
    xaxis_title="dataset /  features",
    yaxis_title="Improvement over using all features",
    legend_title="Estimator",
    width=1200,
    height=600
)
fig.show()